# PUTATIVE WORKFLOW


## WORKFLOW EDITOR PLUGIN
- FINE-TUNE SEGMENTATIONS
  - export workflow.jsons
    - masks:
      - nuclei
      - cellmask
      - cytoplasm
    - organelles:
      - lyso
      - mito
      - golgi
      - perox
      - ER
      - LD


## BATCHPROCESS WORKFLOW
- BATCH PROCESS
  - load workflow.jsons for: 
  1. masks
    - export: masks .tiff as stack (nuclei, cellmask, cytoplasm)
  2. organelles
    - export individual .tiffs



## NOTEBOOK OR ***FUTURE*** PLUGIN
- COLLECT ORGANELLE STATS
  - extract masks.tiffs as individual
    - nuclei, cellmask, cytoplasm
  - collect regionprops for all organelles
    - export .csvs


## NOTEBOOK OR __FUTURE__ PLUGIN
- SUMMARIZE STUDY DATA
  - munge .csv to create summary stats across all cells/images




### IMports


In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json


In [ ]:
import infer_subc_2d.constants

-----------------
## EXTRACT mask stack




_____________

## TO DO
- add "segmentation name" field instead of copying from workflow.json name


- choose alternate conf_XXX.json location. 
  - strategy:  add to "prebuilt" list from path


  
  ## FILE NAME CONVENTIONS

  raw file name is kept.

  PREFIX = "segmentation name" or regionprop name.  e.g. 
  SUFFIX = "description" i.e. 

In [2]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union

import numpy as np
import pandas as pd

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.core.file_io import (read_czi_image,
                                                                    export_inferred_organelle,
                                                                    import_inferred_organelle,
                                                                    export_tiff,
                                                                    list_image_files)

import time
%load_ext autoreload
%autoreload 2



In [6]:
# this will be the example image for testing the pipeline below
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
int_data_path = data_root_path / "intermediate"
im_type = ".tiff"

# get the list of all files
img_file_list = list_image_files(int_data_path,im_type)

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [32]:
im_path = Path(img_file_list[0])
im_path

PosixPath('/Users/ahenrie/Projects/Imaging/data/intermediate/masks_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR1_N23_Unmixed.tiff')

In [52]:
full_name = im_path.name

"/".join(im_path.parts[:-1]+[full_name])

TypeError: can only concatenate tuple (not "list") to tuple

In [20]:
prefix = full_name.split("_")[0]
stem = full_name.lstrip(prefix)
prefix, stem

('masks', '_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR1_N23_Unmixed.tiff')

In [ ]:
seg_type = full_name.split("_")[0]


if seg_type

In [62]:
# MASK process
# 1. get a listof all files based on a "prefix" and "suffix" for a given path
# dump three .tiff from teh mask multichannel tiff
def explode_masks(root_path: Union[Path,str], prefix: str= "masks", postfix: str = ".tiff"):
    """  
    TODO: add loggin instead of printing
        append tiffcomments with provenance
    """
    from tifffile import imwrite, imread#, tiffcomment

    if isinstance(root_path, str): root_path = Path(root_path)
    img_file_list = list_image_files(root_path,im_type, prefix)
    wrote_cnt = 0
    for img_f in img_file_list:
        # load image 
        full_name = Path(img_f).name
        if full_name.startswith(prefix):
            _prefix = full_name.split("_")[0]
            if _prefix != prefix:
                print(f"prefix has underscore!!- {prefix}")
            stem = full_name.lstrip(prefix)
            
            image = imread(img_f)
            assert image.shape[0]==3
            nuclei = image[0]
            cellmask = image[1]
            cytosol = image[2]

            # write wasks
            root = img_f.rstrip(full_name)
            ret1 = imwrite(f"{root}nuclei{stem}", nuclei)
            ret2 = imwrite(f"{root}cellmask{stem}", cellmask)
            ret3 = imwrite(f"{root}cytosol{stem}", cytosol)
            print(f"wrote {{nuclei,cellmask,cytosol}}{stem}")
            wrote_cnt += 1
        else:
            print(f"how thefark!!! {img_f}")

    return wrote_cnt




In [63]:
cnt = explode_masks(int_data_path)
cnt

wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR1_N23_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR2_N03_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR2_N04_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_ZSTACK_PBTOhNGN2hiPSCs_BR3_N14_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR3_N08_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_ZSTACK_PBTOhNGN2hiPSCs_BR3_N01_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR1_N15_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR2_N05_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR2_N02_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR2_N10_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR1_N22_Unmixed.tiff
wrote {nuclei,cellmask,cytosol}_MCZtest_ZSTACK_PBTOhNGN2hiPSCs_BR1_N14_Unmixed.tiff
wrote {n

43

In [ ]:


# for a list of "prefixes"  collect stats + cross stats masked by cytosol (including nuclei masked by cellmask)




def get_stats(int_path: Union[Path,str], out_path: Union[Path, str], organelles: List[str]= ["nuclei","golgi","peroxi"], postfix: str = ".tiff"):
    """  
    TODO: add loggin instead of printing
        append tiffcomments with provenance
    """
    from tifffile import imwrite, imread#, tiffcomment

    # load organelles and masks
    
    if isinstance(int_path, str): int_path = Path(int_path)
    if isinstance(out_path, str): out_path = Path(out_path)
    
    img_file_list = list_image_files(int_path,postfix, "cytosol")

    for img_f in img_file_list:
        # load image 
        full_name = Path(img_f).name
     
        stem = full_name.lstrip(prefix)
        root = img_f.rstrip(full_name)

        # read cytosol    
        cytosol = imread(img_f)
        cellmask = imread(f"{root}cellmask{stem}") 
        
        org_img = [ imread(f"{root}{o}{stem}") for o in organelles]

        # dump_stats]
        nucleus_table = dump_stats("nucleus", nuclei_obj, img_data[NUC_CH], soma_obj, out_data_path, source_file)






# linearly unmixed ".czi" files are here
int_data_path = data_root_path / "intermediate"
im_type = ".tiff"
# save output ".tiff" files here
out_data_path = data_root_path / "out"

get_stats(int_data_path, out_data_path, organelles=["nuclei", "golgi","perox"])



In [66]:
organelles = ["nuclei","golgi","peroxi"]

organelles.count("xxxx")

0